#Regressions

In [26]:
#Libraries that we are goint to use:
import pandas as pd
#!pip install statsmodels

import statsmodels.api as sm
import statsmodels.formula.api as smf


In [29]:
df= pd.read_csv("/Users/polgarcia/Documents/Master/T2/Text_Mining/H1(scrapping+D&D)/TextMininingHW1/final_cleaned_data.csv")

In [30]:
print(df.head())

                                     Name  Barcelona  Price  Rating  stars  \
0                        Isabella's House          1   2142    8.58    4.0   
1                         Sonder Casa Luz          1   4870    8.48    4.0   
2  Axel TWO Barcelona 4 Sup - Adults Only          1   2149    8.38    4.0   
3                           Acta Voraport          1   1540    8.88    3.0   
4                         Tembo Barcelona          1   2840    8.88    4.0   

   num_coments                                        Description  \
0        1.390  Isabella's House se encuentra a 2,9 km de Pase...   
1        2.889  El Sonder Casa Luz se encuentra en Barcelona y...   
2        2.742  El TWO Hotel Barcelona by Axel está situado en...   
3       13.850  Acta Voraport está en Barcelona, a 10 min a pi...   
4        2.697  Tembo Barcelona está en Barcelona, a 16 min a ...   

   During_Treatment                                Cleaned_Description  \
0                 1  isabell hous encuentr

## Explanation of the Dataset

- **`Name`**: The name of the hotel.  
- **`Barcelona`**: This column can take only two values:  
  - `0` if the hotel is not in Barcelona but in our control city, which in this case is Madrid (chosen due to its price similarity).  
  - `1` if the hotel is located in Barcelona.  
- **`Price`**: The price of the hotel in Euros.  
- **`Rating`**: The rating of the hotel on the booking website.  
- **`Description`**: The original hotel description as found on the website.  
- **`During_Treatment`**: This column can take two values:  
  - `0` if the recorded prices are from the control week (two weeks after the event).  
  - `1` if the prices correspond to the week of the event.  
- **`Cleaned_Description`**: The hotel description after text preprocessing.  


## Part 3.a
Write down a fixed effects regression equation that allows you to derive a difference-
in-difference estimate of the effect of the event on prices. Think of controls to
add, why is this relevant? Explain why you need a second city for this.

First lets run the most basic regression:

In [31]:

# Create the interaction term (DiD variable)
df["Barcelona_x_treatment"] = df["Barcelona"] * df["During_Treatment"]

#Fist we will run a regression without CONTROLS.

model = smf.ols(
    "Price ~ During_Treatment + Barcelona + Barcelona_x_treatment + C(Barcelona) + C(During_Treatment)", 
    data=df
).fit()

# Print summary results
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.359
Model:                            OLS   Adj. R-squared:                  0.357
Method:                 Least Squares   F-statistic:                     206.7
Date:                Tue, 04 Feb 2025   Prob (F-statistic):          6.64e-141
Time:                        20:06:01   Log-Likelihood:                -12038.
No. Observations:                1482   AIC:                         2.409e+04
Df Residuals:                    1477   BIC:                         2.411e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept               

What can we take from here: **#The event significantly increases hotel prices in Barcelona (Barcelona_x_treatment = 1476.07, p < 0.001).**

### **Think of controls to add, why is this relevant? Explain why you need a second city for this.**

R: Controls we can  add:
 -  The rating in the web.  
 - The Number of stars of the hotels, as the hotels with more stars tend to have higher price, 

 - The distance to the center. The hotels that are more close tho the event/the center of the city ( the event is not done exacly in the center of the city but in the center are very good comunications to go to the event).

 - The number of coments that each hotel has in the webside .... (add more explanation)


In [32]:
#we will add the number of comments and the stars of the hotels. The distance to the center we didnt add because we had some problems with the Madrid dataset.add

model2 = smf.ols(
    "Price ~ During_Treatment + Barcelona + Barcelona_x_treatment + Rating + stars + num_coments + C(Barcelona) + C(During_Treatment)", 
    data=df
).fit()

# Print summary results
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.493
Model:                            OLS   Adj. R-squared:                  0.491
Method:                 Least Squares   F-statistic:                     238.6
Date:                Tue, 04 Feb 2025   Prob (F-statistic):          4.09e-213
Time:                        20:24:44   Log-Likelihood:                -11849.
No. Observations:                1480   AIC:                         2.371e+04
Df Residuals:                    1473   BIC:                         2.375e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept               

## 3.b


### How would you use text features from the description as controls? Think about the text in the descriptions you scraped. How would this help? Why would termslike "Barcelona" not help?

Use new controls as: 
"piscina" , "wifi","spa", "lujoso"  "cerca del centro" ,"cerca del metro"
More if you wand

In [ ]:
# will need ot do some preprocesessing. Using Results from part 2
vocab, index = {}, 1  # start indexing from 1
dfwords=df["Cleaned_Description"]
for col in dfwords:
    tokens = col.split()
    for token in tokens:
      if token not in vocab:
        vocab[token] = index
        index += 1
        
vocab_stem=vocab
vocab_stem_size = len(vocab)
print(vocab_stem)
print(" ")
print("Total size of stemmed vocabulary:", vocab_stem_size)

vectors=[]
for doc in corpus_data.text_preproc:
    vectors.append(vectorize(doc, vocab_stem))

df = pd.DataFrame(vectors)
df

In [ ]:
#HERE the data frame is not the same that we had because now we will have the columns of the frequency of the data
model2=smf.ols(
    "price ~ During_Treatment + Barcelona + Barcelona_x_treatment + C(Barcelona) + C(During_Treatment) + X + XX + XXX+ XXX",
    data=df
).fit()

- **Why bacelona not help?**  : Almost Every hotel in the barcelona  mention "Barcelona" and making it non-informative. the word Barcelona doesn’t add variation to the model.

[]


## 3.c


Now suppose we want to decompose the treatment effect by hotel quality. Can
you use the text description here? How would you use them to study heterogenous
treatment effects? Write down a regression equation.

WE can add another column that represents the quality. Quality =2 represents Luxury .Quality=1 represens Mid-range,  Quality = 0 represent Budget hotels

Luxury hotels (mentioning "5-star", "spa", "rooftop", etc.).
Mid-range hotels ("comfortable", "good location", "breakfast included").
Budget hotels ("affordable", "hostel", "shared room").

In [ ]:
#need to generate another model where the control columns are some